In [1]:
import glob
import os
import pandas as pd

def get_data(dn):
    data = {
        "news":[],
        "ans":[]
    }
    # 找到所有資料夾
    dirs = glob.glob(os.path.join(dn, "*"))
    # 找到所有檔案
    for d in dirs:
        pat = os.path.join(d, "*.txt")
        fs = glob.glob(pat)     
    # 開啟檔案
        for fp in fs:
            with open(fp, "r", encoding="utf-8") as f:
                news = f.read()
    # 把最後一層的東西拿出來
            ans = os.path.split(d)[-1]
            data["news"].append(news)
            data["ans"].append(ans)
    df = pd.DataFrame(data, columns=["news", "ans"])
    return df
    

In [2]:
tran_df = get_data("chinese_news_trans")
test_df = get_data("chinese_news_test")


In [3]:
kind = tran_df["ans"].unique()

trans = {k:i for i, k in enumerate(kind)}

reverse_trans = {i:k for i, k in enumerate(kind)}
reverse_trans

{0: '交通',
 1: '政治',
 2: '教育',
 3: '環境',
 4: '經濟',
 5: '藝術',
 6: '計算機',
 7: '軍事',
 8: '醫藥',
 9: '體育'}

In [4]:

# unique value count



y_tran = tran_df["ans"].replace(trans)
y_test = test_df["ans"].replace(trans)

y_tran

0       0
1       0
2       0
3       0
4       0
       ..
2632    9
2633    9
2634    9
2635    9
2636    9
Name: ans, Length: 2637, dtype: int64

In [5]:
import jieba

def newscut(s):
    # 用空白表示英文語法
    s = " ".join(jieba.cut(s))
    s = s.replace("\r", "").replace("\n", "")
    return s


tran = tran_df["news"].apply(newscut)
test = test_df["news"].apply(newscut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.693 seconds.
Prefix dict has been built successfully.


In [6]:
# 答案跟資料都要變成數字
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_tran = vec.fit_transform(tran)
x_test = vec.transform(test)
# 測試資料不做fit
print(x_tran)


  (0, 623)	1
  (0, 25511)	2
  (0, 98437)	4
  (0, 38284)	2
  (0, 70743)	2
  (0, 40650)	2
  (0, 34510)	1
  (0, 50393)	1
  (0, 88754)	1
  (0, 45504)	1
  (0, 62908)	1
  (0, 28588)	1
  (0, 81669)	1
  (0, 11988)	1
  (0, 73648)	1
  (0, 89525)	1
  (0, 91723)	1
  (0, 39986)	1
  (0, 86172)	1
  (0, 11904)	1
  (0, 69130)	1
  (0, 40687)	1
  (0, 17738)	1
  (0, 55019)	1
  (0, 18533)	1
  :	:
  (2636, 64330)	1
  (2636, 64336)	1
  (2636, 52999)	1
  (2636, 11655)	1
  (2636, 91857)	1
  (2636, 30612)	1
  (2636, 38027)	1
  (2636, 49671)	1
  (2636, 38036)	4
  (2636, 66775)	2
  (2636, 50730)	1
  (2636, 23436)	1
  (2636, 23460)	1
  (2636, 13626)	1
  (2636, 93726)	1
  (2636, 78546)	1
  (2636, 79029)	1
  (2636, 91911)	1
  (2636, 66770)	1
  (2636, 56539)	1
  (2636, 95153)	2
  (2636, 23123)	1
  (2636, 70733)	1
  (2636, 8321)	1
  (2636, 66769)	1


In [7]:
# 分群

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_tran, y_tran)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [8]:
# 預測答案

from sklearn.metrics import accuracy_score
pre = clf.predict(x_test)
accuracy_score(pre, y_test)

# 太神啦~~~完全正確

1.0

In [9]:
# 輸入介面

s = input("輸入一篇新聞:")
s = vec.transform([newscut(s)])
pre = clf.predict(s)[0]
ans = reverse_trans[pre]
print("應該是：", ans, "類的新聞")

輸入一篇新聞:中職本季首場「金控大戰」，富邦悍將隊洋投索沙吃定中信兄弟隊，先發七局只被揮出四支安打、失一分，最快球速飆到一百五十五公里，寫下個人加盟中職新高，悍將隊終場三比一獲勝，兄弟吞下開季二連敗。  悍將隊總教練洪一中領軍首勝到手，也是生涯第八百八十五勝，仍居中職史上第一。  索沙去年季中轉戰南韓職棒前，對戰兄弟三連勝、連續廿二局未失分，回鍋中職首戰又讓兄弟打線吃足苦頭，昨天在台中洲際球場前四局讓對手掛零，五局下失掉唯一一分，中斷連廿六局無失分紀錄，仍以優質內容奪下新球季首勝。  索沙真是兄弟剋星，生涯對戰五場投出跨季四連勝，合計卅八局只失四分，包括最近廿九局失一分，防禦率只有零點九五，兄弟必須趕快找出破解之道。  悍將隊面對兄弟三名投手，全場揮出七支安打，高國輝三支二，二局上從兄弟洋投德保拉的手中掃出右外野陽春砲，本季首轟成為致勝一擊；林哲瑄七局上揮出今年首安，也是生涯第五百安紀錄。  德保拉先發七局，被揮出五支安打、失兩分，同樣優質先發吞下本季首敗。  中職今天在台中、桃園球場進行兩場比賽，統一獅隊卅八歲老將潘威倫終於在本季初登板前，簽下兩年合約，接下來兩季月薪分別是四十八萬、五十三萬，總值一千二百一十二萬元，另設立激勵獎金。  潘威倫二○○三年展開職棒生涯，兩年新約平均月薪五十萬五千元，歷年收入（含簽約金）超過一億元，成為中職史上第一人，也是首位談到四月才簽約的球員。  潘威倫今天將在桃園球場對決樂天桃猿隊，挑戰生涯第一百四十四勝紀錄，再拚史上新高。
應該是： 體育 類的新聞


In [10]:
# 看其他機率
test = vec.transform([newscut(n)])
proba = clf.predict_proba(test)[0]
proba = list(enumerate(proba))
for i, prod in 


9